In [41]:
import os
import pandas as pd
import numpy as np
import nltk
from tqdm.notebook import tqdm
from datetime import datetime

In [42]:
df_standings = {}
for year in range(2010,2023):
    p1 = str(year)[-2:]
    p2 = str(year+1)[-2:]
    df_standings[year] = pd.read_csv(f'raw_data/transfermarkt/premier_league{p1}-{p2}.csv')

In [43]:
df_games = {}
for year in range(2010,2022):
    p1 = str(year)[-2:]
    p2 = str(year+1)[-2:]
    df_games[year] = pd.read_csv(f'raw_data/football-data/EPL Games {p1}-{p2}.csv').dropna()

In [44]:
df_TM = pd.read_csv('raw_data/transfermarkt/TMValueEngland.csv')

There is an inconsistency in the team names. We could create a teamid for each team and add it to each table, but instead we will normalize the team names across the different tables. Since the table with TMValue has the most teams and their complete names, we will use it as a reference.

In [45]:
teams_TM = np.unique(df_TM['Team'])

In [46]:
teams_standings = np.unique(np.concatenate([df_standings[year]['Team'] for year in range(2010,2023)]))

In [47]:
teams_games = np.unique(np.concatenate([df_games[year]['HomeTeam'] for year in range(2010,2022)]))

In order to find the matches we wrote a simple code to compare the strings. Given teamA and teamB, a match is found if teamA is a substring of teamB or the characters of teamA form a subset of the characters of teamB. Otherwise, we find the match based on the Levenshtein distance. This works for all team names, except for Tottenham, so we just hardcode that exception.

In [48]:
def get_dict_map(teamsA, teamsB):
    dict_map = {}
    for teamA in teamsA:
        best_distance = 1000
        for teamB in teamsB:
            if teamA in teamB:
                answer = teamB
                break
            if set(teamA).issubset(set(teamB)):
                answer = teamB
            temp_distance = nltk.edit_distance(teamA, teamB, transpositions=True)
            if temp_distance<best_distance:
                best_distance = temp_distance
                answer = teamB
        if teamA == 'Spurs':
            dict_map[teamA] = 'Tottenham Hotspur'
        else:
            dict_map[teamA] = answer
    return dict_map

We will store the new tables in the data directory.

In [49]:
os.makedirs('data', exist_ok=True)

In [50]:
for year in range(2010,2023):
    p1 = str(year)[-2:]
    p2 = str(year+1)[-2:]
    df_standings[year].replace({'Team': get_dict_map(teams_standings, teams_TM)}, inplace=True)
    df_standings[year].to_csv(f'data/premier_league{p1}-{p2}.csv', index=False)

In [51]:
for year in range(2010,2022):
    p1 = str(year)[-2:]
    p2 = str(year+1)[-2:]
    df_games[year].replace({'HomeTeam': get_dict_map(teams_games, teams_TM)}, inplace=True)
    df_games[year].replace({'AwayTeam': get_dict_map(teams_games, teams_TM)}, inplace=True)
    if year in [2015, 2017, 2018, 2019, 2020, 2021]:
        df_games[year]['Date'] = pd.to_datetime(df_games[year]['Date'], format="%d/%m/%Y")
    else:
        df_games[year]['Date'] = pd.to_datetime(df_games[year]['Date'], format="%d/%m/%y")
    df_games[year].to_csv(f'data/EPL Games {p1}-{p2}.csv', index=False)

In [52]:
df_TM['Date'] = pd.to_datetime(df_TM['Date'])
df_TM.to_csv('data/TMValueEngland.csv', index=False)

Notice that in the above code, we also convert the date from a string to a timestamp. This makes it easier to compare dates and determine the TMValue for each team at each game. We determine the TMValue based on the most recent known value at the time of the game and also at the beginning of the season.

In [53]:
def add_TMValue(games: pd.DataFrame, dfTMValues: pd.DataFrame, team_key: str, new_key: str):
    entries = []
    for i in range(len(games)):
        date1 = games.iloc[i]['Date']
        diff_days = np.Inf
        closest_date = None
        TMValue = None
        df_temp = dfTMValues[dfTMValues['Team'] == games.iloc[i][team_key]]
        for j in range(len(df_temp)):
            date2 = df_temp.iloc[j]['Date']
            time_interval = date1-date2 
            if date2 < date1:
                if time_interval.days < diff_days:
                    diff_days = time_interval.days
                    closest_date = date2
                    TMValue = df_temp.iloc[j]['TMValue']
        entries.append(TMValue)
    games[new_key] = entries
    return games

def add_TMValue_beginning(games: pd.DataFrame, dfTMValues: pd.DataFrame, team_key: str, new_key: str):
    team_names = np.unique(games['HomeTeam'])
    map_tm_value = {}
    for name in team_names:
        date1 = min(games['Date'])
        diff_days = np.Inf
        closest_date = None
        TMValue = None
        df_temp = dfTMValues[dfTMValues['Team'] == name]
        for j in range(len(df_temp)):
            date2 = df_temp.iloc[j]['Date']
            time_interval = date1-date2 
            if date2 < date1:
                if time_interval.days < diff_days:
                    diff_days = time_interval.days
                    closest_date = date2
                    TMValue = df_temp.iloc[j]['TMValue']
        map_tm_value[name] = TMValue
    entries = []
    for i in range(len(games)):
        entries.append(map_tm_value[games.iloc[i][team_key]])
    games[new_key] = entries
    return games

In [54]:
for year in tqdm(range(2010,2022), desc='Year'):
    p1 = str(year)[-2:]
    p2 = str(year+1)[-2:]
    df_games[year] = add_TMValue(df_games[year], df_TM, 'HomeTeam', 'HTMValue')
    df_games[year] = add_TMValue(df_games[year], df_TM, 'AwayTeam', 'ATMValue')
    df_games[year] = add_TMValue_beginning(df_games[year], df_TM, 'HomeTeam', 'HTMValueSeason')
    df_games[year] = add_TMValue_beginning(df_games[year], df_TM, 'AwayTeam', 'ATMValueSeason')
    df_games[year].to_csv(f'data/EPL Games {p1}-{p2}.csv', index=False)

Year:   0%|          | 0/12 [00:00<?, ?it/s]